In [ ]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
import json

In [ ]:
def fetch_data(year, standing_url):
    all_matches = []
    data = requests.get(standing_url)
    soup = bts(data.text, 'html.parser')
    standing_table = soup.select_one('table.stats_table')
    
    links = [f"https://fbref.com{l.get('href')}" for l in standing_table.find_all('a') if '/squads/' in l.get('href')]
    
    for team_url in links:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = bts(data.text, 'html.parser')
        
        links = [f"https://fbref.com{l.get('href')}" for l in soup.find_all('a') if l.get('href') and "all_comps/shooting/" in l.get('href')]
        
        if len(links) == 0:
            continue
        
        data = requests.get(links[0])
        try:
            shooting = pd.read_html(data.text, match="Shooting")[0]
            shooting.columns = shooting.columns.droplevel()
        except ValueError:
            continue
        
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist","FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)
        
    return pd.concat(all_matches)

years = list(range(2023,2017, -1))
all_data = []

for year in years:
    standing_url = f"https://fbref.com/en/comps/9/{year}-Premier-League-Stats"
    all_data.append(fetch_data(year, standing_url))